# LangChain to prompt programmatically in Python

1. Create an account on [OpenAI Platform](https://platform.openai.com/)
2. Create a project
3. Create an API Key in the project (Settings > Project > API Keys)
4. Paste it into the `.env` file under the variable `OPENAI_API_KEY`

## Load model

In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="chatgpt-4o-latest")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff43d8a4b0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff43dac980>, root_client=<openai.OpenAI object at 0xffff445948c0>, root_async_client=<openai.AsyncOpenAI object at 0xffff43d8a510>, model_name='chatgpt-4o-latest', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompting seralization

### Load markdown template

In [2]:
with open('../prompts/single-day/ES.md', 'r') as file:
    template = file.read()
    
template

'El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.'

### Create prompt object

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    template_format='f-string',
)

prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

### Save prompt to JSON

In [4]:
path = '../prompts/single-day/ES.json'
prompt.save(path)

## Chaining with LLMs

### Define chain

In [5]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff43d8a4b0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff43dac980>, root_client=<openai.OpenAI object at 0xffff445948c0>, root_async_client=<openai.AsyncOpenAI object at 0

### Preprocess input data

In [6]:
ticker = 'AAPL'
freq = 'D'

In [7]:
from modules import utils

r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=1
)

r

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Date
2024-06-11    7.26492
Name: AAPL, dtype: float64

In [8]:
date = r.index[0].strftime('%Y-%m-%d')
change = round(r.values[0], 2)

date, change

('2024-06-11', 7.26)

### Invoke chain with input data

In [9]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

El movimiento extremo del 7.26% en el precio de las acciones de Apple Inc. (ticker: AAPL) el 11 de junio de 2024 puede atribuirse principalmente al entusiasmo del mercado generado tras la presentación de nuevas funciones impulsadas por inteligencia artificial (IA) durante su conferencia anual de desarrolladores, WWDC 2024 (Worldwide Developers Conference), llevada a cabo el 10 de junio de 2024.

Motivo principal: Presentación de funciones de Inteligencia Artificial ("Apple Intelligence")

Durante la WWDC 2024, Apple presentó oficialmente su estrategia basada en inteligencia artificial, denominada “Apple Intelligence”, que integra funciones generativas avanzadas directamente en sus dispositivos y sistemas operativos (iOS 18, iPadOS 18, macOS Sequoia). Estas características incluyen:

- Una versión mejorada de Siri con capacidades generativas alimentadas por modelos de lenguaje de gran tamaño (LLM).
- Integración con ChatGPT (de OpenAI), permitiendo responder preguntas complejas y genera

## Export output to file

In [17]:
with open(f'reports/{ticker}.md', 'w') as file:
    file.write(output)

---

## Change model to enable web search

### Compose chain

In [24]:
model = ChatOpenAI(model="gpt-4o-search-preview")
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff432f7050>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff4310d5b0>, root_client=<openai.OpenAI object at 0xffff432f4d70>, root_async_client=<openai.AsyncOpenAI object at 0

### Invoke chain

In [25]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

El 11 de junio de 2024, las acciones de Apple Inc. (AAPL) experimentaron un notable incremento del 7.26%, alcanzando un precio récord de $207.15 por acción. Este movimiento significativo se atribuye principalmente a los anuncios realizados durante la Conferencia Mundial de Desarrolladores (WWDC) de Apple, celebrada el 10 de junio de 2024.

Durante la WWDC, Apple presentó "Apple Intelligence", una plataforma de inteligencia artificial integrada en iOS 18, y anunció una colaboración con OpenAI para incorporar ChatGPT en sus dispositivos. Estas innovaciones generaron expectativas de un nuevo ciclo de actualización de iPhones, impulsando el optimismo entre los inversores. ([bloomberg.com](https://www.bloomberg.com/news/articles/2024-06-11/apple-hits-first-record-this-year-on-hopes-ai-will-fuel-upgrades?utm_source=openai))

Analistas de JPMorgan destacaron que las mejoras en las aplicaciones nativas y de terceros, junto con la integración de Apple Intelligence, podrían estimular la demanda 

### Export output to file

In [27]:
with open(f'reports/{ticker}_search.md', 'w') as file:
    file.write(output)